In [1]:
import string
import re
import pandas as pd

# Read input data
input_df = pd.read_excel('Input.xlsx')
stop_words = open('Stop_words.txt', encoding='utf-8').read().split()

# Initialize output DataFrame
output_df = pd.DataFrame(columns=[
    'URL_ID',
    'URL',
    'POSITIVE SCORE',
    'NEGATIVE SCORE',
    'POLARITY SCORE',
    'SUBJECTIVITY SCORE',
    'AVG SENTENCE LENGTH',
    'PERCENTAGE OF COMPLEX WORDS',
    'FOG INDEX',
    'AVG NUMBER OF WORDS PER SENTENCE',
    'COMPLEX WORD COUNT',
    'WORD COUNT',
    'SYLLABLE PER WORD',
    'PERSONAL PRONOUNS',
    'AVG WORD LENGTH'
])


In [2]:
def process_text(file_path, stop_words):
    text = open(file_path, encoding='utf-8').read()
    lower_case = text.lower()
    cleaned_text = lower_case.translate(str.maketrans('', '', string.punctuation))
    tokenized_words = cleaned_text.split()

    final_words = [word for word in tokenized_words if word not in stop_words]
    
    return final_words


In [3]:
def neg_pos(file_path, final_words):
    positive_score = 0
    negative_score = 0

    with open('positive-words.txt', 'r') as file:
        for line in file:
            clear_line = line.replace('\n', '').replace(',', '').replace("'", '').strip()
            if clear_line in final_words:
                positive_score += 1

    with open('negative-words.txt', 'r') as file:
        for line in file:
            clear_line = line.replace('\n', '').replace(',', '').replace("'", '').strip()
            if clear_line in final_words:
                negative_score += 1

    positive_score += 0.000001
    negative_score += 0.000001

    polarity_score = (positive_score - negative_score) / (positive_score + negative_score)
    subjectivity_score = (positive_score + negative_score) / (len(final_words) + 0.000001)

    print("Positive Score:", positive_score)
    print("Negative Score:", negative_score)
    print("Polarity Score:", polarity_score)
    print("Subjectivity Score:", subjectivity_score)
    
    return positive_score, negative_score, polarity_score, subjectivity_score


In [4]:
def count_syllables(word):
    word = word.lower()
    if word.endswith(('es', 'ed')) and len(word) > 2:
        return 1
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith('e') and not word.endswith('le'):
        count -= 1
    if count == 0:
        count = 1
    return count


In [5]:
def calculate_fog_index(text):
    sentences = re.split(r'[.!?]', text)
    num_sentences = len(sentences)

    words = re.findall(r'\b\w+\b', text)
    num_words = len(words)
    num_complex_words = 0
    num_syllables = 0
    personal_pronouns = ['i', 'me', 'my', 'mine', 'myself', 'you', 'your', 'yours', 'yourself', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'we', 'us', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourselves', 'they', 'them', 'their', 'theirs', 'themselves']

    for word in words:
        syllables = count_syllables(word)
        num_syllables += syllables
        if syllables > 2:
            num_complex_words += 1

    # Calculate Average Sentence Length
    average_sentence_length = num_words / num_sentences
    
    # Calculate Percentage of Complex Words
    percentage_complex_words = (num_complex_words / num_words) * 100
    
    # Calculate Fog Index
    fog_index = 0.4 * (average_sentence_length + percentage_complex_words)
    
    # Calculate Average Number of Words per Sentence
    avg_words_per_sentence = num_words / num_sentences
    
    # Calculate Average Syllables per Word
    avg_syllables_per_word = num_syllables / num_words
    
    # Calculate Personal Pronouns Count
    pronoun_count = sum(1 for word in words if word.lower() in personal_pronouns)
    
    # Calculate Average Word Length
    avg_word_length = sum(len(word) for word in words) / num_words
    
    return (
        average_sentence_length,
        percentage_complex_words,
        fog_index,
        avg_words_per_sentence,
        num_complex_words,
        num_words,
        avg_syllables_per_word,
        pronoun_count,
        avg_word_length
    )


In [6]:
# Loop through file names from blackassign0001 to blackassign0100
for i in range(1, 101):
    file_path = f'blackassign{str(i).zfill(4)}_output.txt'
    
    final_words = process_text(file_path, stop_words)
    print("Results for article", i)
    
    positive_score, negative_score, polarity_score, subjectivity_score = neg_pos(file_path, final_words)

    with open(file_path, encoding='utf-8') as file:
        text = file.read()

    (
        avg_sentence_length,
        percentage_complex_words,
        fog_index,
        avg_words_per_sentence,
        num_complex_words,
        num_words,
        avg_syllables_per_word,
        pronoun_count,
        avg_word_length
    ) = calculate_fog_index(text)
    
    # Get the URL for the current article from the Input.xlsx file
    current_url = input_df.iloc[i - 1]['URL']
    
    # Append the results to the output DataFrame
    output_df = output_df.append({
        'URL_ID': f'blackassign{str(i).zfill(4)}',
        'URL': current_url,
        'POSITIVE SCORE': positive_score,
        'NEGATIVE SCORE': negative_score,
        'POLARITY SCORE': polarity_score,
        'SUBJECTIVITY SCORE': subjectivity_score,
        'AVG SENTENCE LENGTH': avg_sentence_length,
        'PERCENTAGE OF COMPLEX WORDS': percentage_complex_words,
        'FOG INDEX': fog_index,
        'AVG NUMBER OF WORDS PER SENTENCE': avg_words_per_sentence,
        'COMPLEX WORD COUNT': num_complex_words,
        'WORD COUNT': num_words,
        'SYLLABLE PER WORD': avg_syllables_per_word,
        'PERSONAL PRONOUNS': pronoun_count,
        'AVG WORD LENGTH': avg_word_length
    }, ignore_index=True)

# Save the DataFrame to an Excel file
output_df = output_df.sort_values(by='URL_ID')
output_df.to_excel('output.xlsx', index=False)


Results for article 1
Positive Score: 19.000001
Negative Score: 5.000001
Polarity Score: 0.5833332847222262
Subjectivity Score: 0.041884820171230686


C:\Users\dell\AppData\Local\Temp\ipykernel_1976\2233029045.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({


Results for article 2
Positive Score: 41.000001
Negative Score: 24.000001
Polarity Score: 0.2615384534911245
Subjectivity Score: 0.08054523162262052


C:\Users\dell\AppData\Local\Temp\ipykernel_1976\2233029045.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({


Results for article 3
Positive Score: 28.000001
Negative Score: 17.000001
Polarity Score: 0.2444444335802474
Subjectivity Score: 0.07086614477028953


...

C:\Users\dell\AppData\Local\Temp\ipykernel_1976\2233029045.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({


Results for article 97
Positive Score: 14.000001
Negative Score: 27.000001
Polarity Score: -0.3170731552647242
Subjectivity Score: 0.09382151465944734


C:\Users\dell\AppData\Local\Temp\ipykernel_1976\2233029045.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({
C:\Users\dell\AppData\Local\Temp\ipykernel_1976\2233029045.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({


Results for article 98
Positive Score: 5.000001
Negative Score: 3.000001
Polarity Score: 0.24999993750001562
Subjectivity Score: 0.032520333201136856
Results for article 99
Positive Score: 1e-06
Negative Score: 1e-06
Polarity Score: 0.0
Subjectivity Score: 2.2222219753086696e-07
Results for article 100
Positive Score: 1.000001
Negative Score: 1e-06
Polarity Score: 0.9999980000040001
Subjectivity Score: 0.11111132098763099


C:\Users\dell\AppData\Local\Temp\ipykernel_1976\2233029045.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({
C:\Users\dell\AppData\Local\Temp\ipykernel_1976\2233029045.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({
